## Konkordanser for NAOB mot Bokhylla

Versjon: **August 2020**
Inneholder korpus med to utvidelser, en fra 2019 og en fra 2020.

Velg `Run all` fra Cell-menyen over. 

Søkeord er forskjellig med stor og liten forbokstav. Forfatter og tittel kan trunkeres, for eksempel *Agnar Mykle* kan skrives som *mykl*. 

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import pandas as pd
import requests
import inspect
import csv
from IPython.display import HTML


def get_urnkonk(word, params=None, nøkkel = None):
    if params is None:
        params = {}

    para = params
    para['word']= word
    r = requests.post('https://api.nb.no/ngram/urnkonk', json = para)
    rows = ""
    data = r.json()
    if nøkkel != None:
        if nøkkel == 'before':
            try:
                data.sort(key=lambda x: x[nøkkel].split()[-1])
            except:
                True
        else:
            try:
                data.sort(key=lambda x: x[nøkkel])
            except:
                True
    for x in data:
        rows += """<tr>
            <td>
                <a href='{urn}?searchText={kw}' target='_blank' style='text-decoration:none'>{urnredux}</a>
            </td>
            <td>{b}</td>
            <td>{w}</td>
            <td style='text-align:left'>{a}</td>
        </tr>\n""".format(kw=word,
                          urn=x['urn'],
                          urnredux="{t}, {f}, {y}".format(t=x['title'], f=x['author'], y=x['year']),
                          b=x['before'],
                          w=x['word'],
                          a=x['after']
                         )
    res = """<table>{rows}</table>""".format(rows=rows)
    res = HTML(res)
    return res, data

def refine_book_urn(urns = None, words = None, author = None, 
             title = None, ddk  = None, subject = None, period=(1100, 2020), gender=None, lang = None, trans= None, limit=20 ):

    """Refine URNs for books with metadata"""
    
    # if empty urns nothing to refine
    
    if urns is None or urns == []:
        return []
    
    # check if urns is a metadata list, and pick out first elements if that is the case
    if isinstance(urns[0], list):
        urns = [x[0] for x in urns]
        
    frame = inspect.currentframe()
    args, _, _, values = inspect.getargvalues(frame)
    query = {i:values[i] for i in args if values[i] != None and i != 'period' and i != 'urns'}
    query['year'] = period[0]
    query['next'] = period[1] - period[0]
    #print(query)
    return refine_urn(urns, query)

def refine_urn(urns, metadata=None):
    """Refine a list urns using extra information"""
    if metadata is None:
        metadata = {}
    metadata['urns'] = urns
    if not ('words' in metadata):
        metadata['words'] = []
    if not ('next' in metadata or 'neste' in metadata):
        metadata['next'] = 520
    if not 'year' in metadata:
        metadata['year'] = 1500
    r = requests.post('https://api.nb.no/ngram/refineurn', json=metadata)
    return r.json()

def get_urns_from_text(document):
    """Find all URNs in a text-file"""
    
    import re

    with open(document) as fp:
        text = fp.read()
    #print(text)
    return re.findall("[0-9]{13}", text)

def code_toggle(button_text = "klikk"):
    from IPython.display import HTML, display

    display(
        HTML(
        '''<div>
                <style>
                 .mybutton {
                    background-color: white;
                    border: none;
                    color: white;
                    padding: 1px 1px;
                    text-align: center;
                    text-decoration: none;
                    display: inline-block;
                    font-size: 8px;
                    margin: 2px 1px;
                    cursor: pointer;
                }
            </style>
            <script>
                code_show=true; 
                function code_toggle() {
                 if (code_show){
                 $('div.input').hide();
                 } else {
                 $('div.input').show();
                 }
                 code_show = !code_show
                } 
                $( document ).ready(code_toggle);
            </script>
            <form  action="javascript:code_toggle()">
                <input class='mybutton' type="submit" value=''' + '"'  + button_text + '"' + '''>
            </form>
        </div>'''
        ))

In [8]:
code_toggle()

In [9]:
res = dict()


naob = get_urns_from_text("./naob_korpus.txt")
naobutvid = get_urns_from_text("./Litterære kilder tilleggsliste til UBB og NB 20190328.txt")
naobutvid2 = get_urns_from_text("./NAOBTilleggURN2020-06-10.txt")
NAOB = naob + naobutvid + naobutvid2
naob_korpus = {
    'naob' : NAOB,
    'naob original':naob,
    'utvidelse 2019': naobutvid,
    'utvidelse 2020': naobutvid2
}

In [10]:
def sorted_frame(df, sort_array = None):
    if sort_array != None:
        df = df.sort_values(by=sort_array)
    rows = ""
    for x in df.index:
         rows += """<tr>
         <td><a href='{urn}?searchText={word}' target='_'>{author}, {title}, {year}</a></td>
         <td>{before}</td>
         <td>{word}</td>
         <td style='text-align:left'>{after}</td>
         </tr>""".format(
             urn = r.loc[x]['urn'], 
             title = r.loc[x]['title'], 
             author = r.loc[x]['author'],
             year = r.loc[x]['year'],
             before = r.loc[x]['before'],
             word = r.loc[x]['word'],
             after = r.loc[x]['after']
         )
    table = """<table>{rows}</table>""".format(rows = rows)
    return table
            

In [20]:
def konk(søkeord='titt-tei', 
         forfatter ='', 
         tittel ='', 
         periode = widgets.IntRangeSlider(min=1800, max= 2015, value=[1950, 2000]), 
         sorter = widgets.Dropdown(
            options=['year', 'after','before'],
            value='year',
            #rows=3,
            description='sortering',
            disabled=False
         ),
         korpusvalg = widgets.Dropdown(
            options = list(naob_korpus.keys()),
            value=  list(naob_korpus.keys())[0],
            #rows=3,
            description='korpusvalg',
            disabled=False
         ),
         før = (0,12), 
         etter = (0,12),  
         grense = (0,10),
         filnavn = widgets.Text(
            value='',
            placeholder='filnavn for lagring',
            description='fil',
            disabled=False
)
        ):
    if forfatter == '':
        forfatter = "%"
    if tittel == '':
        tittel = "%"
    #urner = refine_book_urn(urns = naob_korpus[korpusvalg], words=[søkeord], author = "%{f}%".format(f=forfatter) , title= "%{t}%".format(t=tittel), period=periode)
    urner = naob_korpus[korpusvalg]
    rhtml, rdata = get_urnkonk(
        søkeord, 
        {
            'urns': [u[0] for u in urner],
            'corpus':'bok', 
            'before':før, 
            'after':etter, 
            'size':grense},
        nøkkel = sorter
    )
    if filnavn != '':
        with open(filnavn, 'a', encoding='utf-8') as the_file:
            fieldnames = ['urn', 'author', 'title', 'year', 'before', 'word', 'after']
            writer = csv.DictWriter(the_file, fieldnames=fieldnames)
            for row in rdata:
                writer.writerow(row)
    return rhtml


im = interact_manual(konk);
im.widget.children[10].description = 'Hent konkordanser!'

interactive(children=(Text(value='titt-tei', description='søkeord'), Text(value='', description='forfatter'), …

In [21]:
interact_manual(konk)

interactive(children=(Text(value='titt-tei', description='søkeord'), Text(value='', description='forfatter'), …

<function __main__.konk(søkeord='titt-tei', forfatter='', tittel='', periode=IntRangeSlider(value=(1950, 2000), description='periode', max=2015, min=1800), sorter=Dropdown(description='sortering', options=('year', 'after', 'before'), value='year'), korpusvalg=Dropdown(description='korpusvalg', options=('naob', 'naob original', 'utvidelse 2019', 'utvidelse 2020'), value='naob'), før=(0, 12), etter=(0, 12), grense=(0, 10), filnavn=Text(value='', description='fil', placeholder='filnavn for lagring'))>